## A Mood Based Movie Recommendation System Using Machine Learning 

### Step 1: Import Required Libraries

In [1]:
import pandas as pd               # for data handling
import numpy as np                # for numerical operations
import ast                        # to convert string lists to real Python lists
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report


### Step 3: Load and Merge Datasets 
#### We’ll use TMDB datasets

In [2]:
movies = pd.read_csv(r"C:\Users\Asus\Downloads\dataset\movie dataset\tmdb_5000_movies.csv")
credits = pd.read_csv(r"C:\Users\Asus\Downloads\dataset\movie dataset\tmdb_5000_credits.csv")

# Merge both datasets using movie ID
movies = movies.merge(credits, left_on='id', right_on='movie_id')


### Step 4: Data Cleaning 

In [3]:
# Keep only important columns
movies = movies[['title_x', 'genres', 'overview', 'cast', 'crew']]
movies.columns = ['title', 'genres', 'overview', 'cast', 'crew']

#Drop Missing Values
movies.dropna(inplace=True)
print("Remaining after dropna:", movies.shape)

#Remove Duplicates
movies.drop_duplicates(inplace=True)


Remaining after dropna: (4800, 5)


### Step 4: Data Processing 

In [4]:
# Convert genres JSON-like strings to list of genre names

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L

movies['genres'] = movies['genres'].apply(convert)
movies['tags'] = movies['overview'] + ' ' + movies['genres'].astype(str)


 ### Step 5: PANAS-Based Mood Input (Positive and Negative Affect Schedule)
 #### Ask the user to enter their PANAS mood scores

In [42]:
positive_score = int(input("Enter your Positive Affect score (0–50): "))
negative_score = int(input("Enter your Negative Affect score (0–50): "))

mood = "good" if positive_score >= negative_score else "bad"
print("Detected Mood:", mood)


Enter your Positive Affect score (0–50):  30
Enter your Negative Affect score (0–50):  35


Detected Mood: bad


### Step 6: Filter Movies Based on Mood

In [43]:
good_mood_genres = ['Comedy', 'Romance', 'Drama', 'Animation', 'Fantasy', 'Science Fiction', 'Music']
bad_mood_genres = ['Drama', 'Romance', 'Documentary', 'Fantasy', 'Music']

def filter_by_mood(mood):
    genres = good_mood_genres if mood == 'good' else bad_mood_genres
    return movies[movies['genres'].apply(lambda x: any(g in x for g in genres))]

filtered_movies = filter_by_mood(mood)


### Step 7: Cosine Similarity for calulate similarity  

In [44]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(filtered_movies['tags'].astype(str)).toarray()

similarity = cosine_similarity(vectors)


### Step 8: Training and Testing 

In [45]:
movies['mood'] = movies['genres'].apply(lambda x: 'good' if any(i in good_mood_genres for i in x) else 'bad')

X = cv.fit_transform(movies['tags'].astype(str)).toarray()
y = movies['mood']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#Support Vector Machine (SVM)
svm_model = SVC(kernel='linear', C=10)  # Linear SVM
svm_model.fit(X_train, y_train)
svm_pred = svm_model.predict(X_test)

# SVM Accuracy
svm_accuracy = accuracy_score(y_test, svm_pred) * 100
print("SVM Accuracy: {:.2f}%".format(svm_accuracy))
print(classification_report(y_test, svm_pred))



SVM Accuracy: 99.67%
              precision    recall  f1-score   support

         bad       0.99      0.99      0.99       201
        good       1.00      1.00      1.00       999

    accuracy                           1.00      1200
   macro avg       1.00      0.99      0.99      1200
weighted avg       1.00      1.00      1.00      1200



#### Naive Bayes (Multinomial) 

In [46]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
nb_pred = nb_model.predict(X_test)

nb_accuracy = accuracy_score(y_test, nb_pred) * 100
print("Naive Bayes Accuracy: {:.2f}%".format(nb_accuracy))
print(classification_report(y_test, nb_pred))

Naive Bayes Accuracy: 92.17%
              precision    recall  f1-score   support

         bad       0.82      0.69      0.75       201
        good       0.94      0.97      0.95       999

    accuracy                           0.92      1200
   macro avg       0.88      0.83      0.85      1200
weighted avg       0.92      0.92      0.92      1200



#### Decision Tree Classifier

In [47]:
dt_model = DecisionTreeClassifier(criterion='gini')
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_test)

dt_accuracy = accuracy_score(y_test, dt_pred) * 100
print("Decision Tree Accuracy: {:.2f}%".format(dt_accuracy))
print(classification_report(y_test, dt_pred))

Decision Tree Accuracy: 99.42%
              precision    recall  f1-score   support

         bad       0.98      0.99      0.98       201
        good       1.00      1.00      1.00       999

    accuracy                           0.99      1200
   macro avg       0.99      0.99      0.99      1200
weighted avg       0.99      0.99      0.99      1200



### Step 9:  Recommend Similar Movies

In [48]:
def recommend(title):
    index = filtered_movies[filtered_movies['title'] == title].index[0]
    distances = list(enumerate(similarity[index]))
    movies_list = sorted(distances, key=lambda x: x[1], reverse=True)[1:11]
    for i in movies_list:
        print(filtered_movies.iloc[i[0]].title)

# Choose a random movie based on mood
random_title = filtered_movies.sample(1).title.values[0]
print("Random Movie Based on Mood:", random_title)
recommend(random_title)


Random Movie Based on Mood: White House Down
Monster House
The Brave Little Toaster
Army of Darkness
Alvin and the Chipmunks: Chipwrecked
The Croods
The Smurfs
And So It Goes
Savva. Heart of the Warrior
Herbie Fully Loaded
Thunder and the House of Magic
